In [ ]:
import pandas as pd
import requests
import json
import datetime
import concurrent.futures
import swifter
import os

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([ codigo, nome, data_hora])
    return lst_movimentos_final

def process_movimento(movimento):
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')
    if data_hora:
        data_hora = converte_data(data_hora)
    return [codigo, nome, data_hora]

def gera_lista_movimentos_multithread(movimentos):
    lst_movimentos_final = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_movimento, movimentos))
        lst_movimentos_final.extend(results)
    return lst_movimentos_final

In [ ]:
def lista_para_dataframe(dados_dict):
  processos = []
  for processo in dados_dict['hits']['hits']:
    numero_processo = processo['_source']['numeroProcesso']
    grau = processo['_source']['grau']
    classe = processo['_source']['classe']['nome']
    try:
      assuntos = processo['_source']['assuntos'] # Pode ter mais de um
    except:
      assuntos = []
    data_ajuizamento = processo['_source']['dataAjuizamento']
    ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
    #formato = processo['_source']['formato']['nome']
    codigo = processo['_source']['orgaoJulgador']['codigo']
    orgao_julgador = processo['_source']['orgaoJulgador']['nome']
    municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
    sort = processo['sort'][0]
    try:
      movimentos = processo['_source']['movimentos']
    except:
      movimentos = []

    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, \
                      codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

  df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                        'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])
  df['movimentos'] = df['movimentos'].swifter.apply(gera_lista_movimentos_multithread)
  df['assuntos'] = df['assuntos'].swifter.apply(gera_lista_assuntos)
  #df['movimentos'] = df['movimentos'].swifter.apply(gera_lista_movimentos_multithread)
  df['data_ajuizamento'] = df['data_ajuizamento'].swifter.apply(converte_data)
  df['ultima_atualizacao'] = df['ultima_atualizacao'].swifter.apply(converte_data)
  try:
    df['movimentos']= df['movimentos'].swifter.apply(lambda x: sorted(x, key=lambda tup: tup[2], reverse=False))
  except:
    pass
  return df

In [ ]:
def criar_dataset(codigo_orgao, data, tamanho_consulta):
  
  df_tjrn = pd.DataFrame()
  url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjrn/_search"
  api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública
  tribunal = 'TJRN'
  orgaoJulgador = codigo_orgao
  size = tamanho_consulta
  data = data

  payload = json.dumps(
  {
  "size": tamanho_consulta,
  "query": {
      "bool": {
        "must": [
            {"match": {"tribunal": tribunal}},
            {"match": {"orgaoJulgador.codigo": orgaoJulgador}},
            {"range": {"dataAjuizamento": {"gte": data }}}
        ]
      }
  },
    "sort": [{"@timestamp": {"order": "asc"}}]
  })

  headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
  dados_dict = response.json() # <class 'dict'>
  df_tjrn = lista_para_dataframe(dados_dict)
  numero_processos = size

  while numero_processos == size:
    numero_processos = len(dados_dict['hits']['hits'])
    tamanho_dicionario_retornado = len(dados_dict['hits']['hits'])-1
    if tamanho_dicionario_retornado < 1:
      print(f'Tamanho do dicionário da página anterior: {tamanho_dicionario_retornado}')
      break
    ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort'][0]
    #print(f'Partindo da posição: {ultima_posicao_dicionario}')
    payload = json.dumps(
    {
    "size": tamanho_consulta,
    "query": {
        "bool": {
          "must": [
            {"match": {"tribunal": tribunal}},
            {"match": {"orgaoJulgador.codigo": orgaoJulgador}},
            {"range": {"dataAjuizamento": {"gte": data}}}
            
          ]
        }
    },
      "search_after": [ ultima_posicao_dicionario ],
      "sort": [{"@timestamp": {"order": "asc"}}]
    })

    headers = {
      'Authorization': api_key,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
    dados_dict = response.json() # <class 'dict'>
    numero_processos = len(dados_dict['hits']['hits'])
    ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort']
    df_tjrn = pd.concat([df_tjrn, lista_para_dataframe(dados_dict)])
    #print(df_tjrn.head(2))
    df_tjrn = df_tjrn[df_tjrn['codigo'] == orgaoJulgador]
    tamanho_dataset = len(df_tjrn.index)
    ultima_data_ajuizamento = dados_dict['hits']['hits'][len(dados_dict['hits']['hits'])-1]['_source']['dataAjuizamento']
    print(f'{datetime.datetime.now()}\t Serventia: {orgaoJulgador}\tNúmero de processos: {tamanho_dataset} \t Data do último processo adicionado: {ultima_data_ajuizamento}' )
    #if tamanho_dataset > 2000000:
    #  break

  try:
    print(f'Número de processos incorporados: {tamanho_dataset}')
  except:
    print(f'Última página do dicionário veio vazia')
  
  return df_tjrn

In [ ]:
def salvar_dataset(df, orgao, data):        
    if not type(orgao) is str:
        str(orgao)
    orgao = orgao.replace(' ', '_')
    nome_df = 'TJRN_' + orgao + '_' + data + '.csv'
    nome_df = nome_df.replace(" ", "_")
    if os.path.isdir('./dados'):
        os.chdir('dados')
    df.to_csv(nome_df, sep=';', header=True, index=False, compression='zip')

In [15]:
#lista_orgaos = ['2º JUIZADO ESPECIAL DA FAZENDA PÚBLICA', '3º JUIZADO ESPECIAL DA FAZENDA PÚBLICA', 
#                '5º JUIZADO ESPECIAL DA FAZENDA PÚBLICA']
orgao = '4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA'
lista_codigo_orgaos = [17460, 17461, 17462, 17463, 17464]
data = '2017-01-01'
for orgao in lista_codigo_orgaos:
    nome_dataset = nome_df = 'TJRN_' + str(orgao) + '_' + data + '.csv'
    nome_dataset = nome_dataset.replace(" ", "_")
    nome_dataset = 'dados/' + nome_dataset
    if os.path.isfile(nome_dataset):
        print('Dataset já existente: ' + nome_dataset)
    else:
        df = criar_dataset(orgao, data, 5000)
        df.reset_index(drop=True, inplace=True)
        salvar_dataset(df, str(orgao), data)

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/1116 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1116 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1116 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1116 [00:00<?, ?it/s]

2024-05-28 19:21:08.661181	 Serventia: 17461	Número de processos: 26116 	 Data do último processo adicionado: 2020-12-09T15:46:47.000Z
Número de processos incorporados: 26116
Tamanho do dicionário da página anterior: -1
Última página do dicionário veio vazia
Tamanho do dicionário da página anterior: -1
Última página do dicionário veio vazia
Tamanho do dicionário da página anterior: -1
Última página do dicionário veio vazia


In [16]:
print((df['orgao_julgador'][0]))

KeyError: 0